In [119]:
import sys
sys.path.append('../src')

import importlib
import numpy as np
from sklearn.covariance import empirical_covariance

import policies
importlib.reload(policies)

<module 'policies' from '/Users/ira/Desktop/BB-portfolios/optimized-ensembling/testing/../src/policies.py'>

# Variance Constrained Policy Testing

In [142]:
importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

alpha = 0.5
policy = policies.VarianceConstrained(pred_dim, model, 0.1, alpha, ys)
out = policy.run_given_preds(ys)

Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver status: optimal
Solver stat

In [143]:
out

array([[-3.56096043e-11,  7.92948256e-12,  5.46953307e-11,
         1.00000000e+00,  3.57434830e-11,  3.76119421e-11],
       [-1.10052815e-12,  3.00033748e-01,  5.07196620e-11,
         5.67460337e-12,  6.99966251e-01,  4.67568846e-11],
       [-1.33751588e-11,  1.57964162e-11, -2.41439631e-12,
         1.00000000e+00,  2.27760864e-12,  1.15550828e-11],
       [-5.50824162e-11,  6.87246943e-10,  6.99209456e-11,
         1.83253271e-10,  6.17781419e-11,  9.99999998e-01],
       [-7.85965307e-12,  2.62328809e-10,  2.99122747e-01,
         3.90756480e-11,  7.00877252e-01,  7.20699841e-11],
       [-2.55596373e-11,  3.66679151e-12,  9.99999998e-01,
         2.09990877e-11, -6.20617770e-12,  5.16110491e-11],
       [ 3.53783837e-01, -3.75538648e-11, -3.70005387e-11,
        -2.49109547e-11,  6.46216163e-01, -3.67859080e-11],
       [ 4.70903707e-11,  4.57901267e-11,  3.45737673e-11,
         2.98306045e-01,  7.01693954e-01,  3.86707159e-11],
       [ 3.58888767e-12,  2.48898955e-11,  4.201

Sanity check: 

1. Does each row sum to 1? Yes! Or at least, up to a tolerance, since there will be small floating point errors.

In [166]:
tolerance = 1e-3

print(np.sum(out, axis=1))
print("Violations: ", n-sum((np.isclose(np.sum(out, axis=1), np.ones(50), atol=tolerance))))

[1.00000000e+00 1.00000000e+00 1.00000000e+00 9.99999999e-01
 1.00000000e+00 9.99999998e-01 1.00000000e+00 1.00000000e+00
 1.00000000e+00 9.99999999e-01 2.43731740e-09 9.99999997e-01
 1.00000000e+00 9.99999999e-01 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 9.99999999e-01
 9.99999998e-01 1.00000000e+00 9.99999999e-01 9.99999999e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 6.99446838e-01
 1.00000000e+00 1.00000000e+00 9.99999999e-01 9.99999999e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 9.99999999e-01 4.56332668e-09 1.00000000e+00
 1.00000000e+00 9.99999986e-01 9.99999999e-01 1.00000000e+00
 1.00000000e+00 1.00000000e+00 9.99999000e-01 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
Violations:  3


2. Is each constraint bounded by 0 and 1?

In [167]:
print(f"Number of allocations greater than 1: {np.sum(out - tolerance > 1)}")
print(f"Number of allocations less than 0: {np.sum(out+tolerance < 0)}")

Number of allocations greater than 1: 0
Number of allocations less than 0: 0


3. Are the variance conditions being approx satisfied? I wrote yes here before but it seems like no?

In [168]:
cov = empirical_covariance(ys, assume_centered=False)

viol = np.zeros(len(out))
for i in range(len(out)):
    viol[i] = np.matmul(np.matmul(out[i], cov), np.transpose(out[i]))

print(f"Max violation: {alpha}")
print(f"Number of variance constraints which violate the max allowed variance: {sum(viol > alpha+0.1)}")

Max violation: 0.5
Number of variance constraints which violate the max allowed variance: 0


In [169]:
viol = np.zeros(len(out))
for i in range(len(out)):
    viol[i] = np.matmul(np.matmul(out[i], cov), np.transpose(out[i]))
viol

array([1.09129128e-01, 5.00000000e-01, 1.09129128e-01, 3.13415176e-03,
       5.00000000e-01, 1.79194875e-03, 4.99999998e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000001e-01, 1.08558548e-20, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 1.09129128e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 1.09129128e-01, 5.00000000e-01, 5.00000001e-01,
       3.13415176e-03, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 1.09129128e-01, 5.00000000e-01, 5.00000000e-01,
       1.04122387e-03, 5.00000000e-01, 6.21221982e-19, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 1.09129128e-01,
       5.00000000e-01, 5.00000000e-01, 3.13414551e-03, 5.00000000e-01,
       1.09129128e-01, 1.09129128e-01])

3. Is the covariance matrix actually measuring the correct thing? A sanity check that the diagonal of the matrix is equal to empirical variance.


In [161]:
print(f"Variances of each coordinate of the ys: \n {np.var(ys, axis=0)}")

print(f"Diagonal of the covariance matrix: \n {np.diagonal(cov)}")

Variances of each coordinate of the ys: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]
Diagonal of the covariance matrix: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]


4. Are all the constraints convex?

If C is positive semi-definite, then xCx^T <= val is a convex constraint on R^n. Maybe something went wrong in calculation of C? 

If the eigenvalues of C are positive, then C must be positive semi-definite. Here we see this is the case:

In [162]:
np.linalg.eigvals(cov) > 0

array([ True,  True,  True,  True,  True,  True])

In [170]:
min(np.linalg.eigvals(cov))


0.0008419627816976109

In [151]:
cov

array([[ 8.27988830e-01, -6.55756916e-03, -8.96512812e-03,
         5.57804785e-02, -6.65418545e-02,  1.18359051e-02],
       [-6.55756916e-03,  1.04122387e-03, -1.09939826e-04,
        -8.67002703e-04, -1.99234699e-03,  4.18234486e-04],
       [-8.96512812e-03, -1.09939826e-04,  1.79194876e-03,
         6.11696668e-04, -5.26528595e-03, -3.28803440e-04],
       [ 5.57804785e-02, -8.67002703e-04,  6.11696668e-04,
         1.09129128e-01, -3.08836710e-02, -1.93356657e-03],
       [-6.65418545e-02, -1.99234699e-03, -5.26528595e-03,
        -3.08836710e-02,  1.02202326e+00, -1.97332997e-02],
       [ 1.18359051e-02,  4.18234486e-04, -3.28803440e-04,
        -1.93356657e-03, -1.97332997e-02,  3.13415178e-03]])